In [ ]:
!pip install -U langchain-text-splitters

In [ ]:
!pip install sentence-transformers==2.7.0

In [ ]:
!pip install numpy==1.26.4

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 20.6 MB/s eta 0:00:00


In [ ]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 8.0 MB/s eta 0:00:00


In [ ]:
documents = [
    "Artificial Intelligence is the simulation of human intelligence by machines.",
    "Machine Learning is a subset of AI that enables systems to learn from data.",
    "Deep Learning uses neural networks with many layers.",
    "Natural Language Processing allows machines to understand human language."
]


from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from groq import Groq

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings
doc_embeddings = model.encode(documents)

# Convert to numpy
doc_embeddings = np.array(doc_embeddings).astype("float32")
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)


print("Number of documents indexed:", index.ntotal)


def retrieve_docs(query, top_k=2):
    query_embedding = model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    return [documents[i] for i in indices[0]]


client = Groq(api_key="")



def generate_answer(query):
    retrieved_docs = retrieve_docs(query)

    context = "\n".join(retrieved_docs)

    prompt = f"""
    Use the context below to answer the question.

    Context:
    {context}

    Question:
    {query}
    """


    response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2
)


    return response.choices[0].message.content
question = "What is Machine Learning?"
answer = generate_answer(question)
print(answer)




Number of documents indexed: 4
Based on the provided context, Machine Learning is a subset of AI that enables systems to learn from data.
